In [1]:
import pandas as pd
import numpy as np

# repo imports
import sys
sys.path.append('..')
from utils import RidgeEnsemble

In [2]:
# from CNN, these match the cluster_data.csv latitudes and longitudes one-to-one
feats = np.load('../cnn/predicting-poverty-replication/cluster_feats.npy')

In [3]:
clusters = pd.read_csv('../LSMS/output/malawi/clusters/cluster_data.csv')

In [4]:
clusters.head()

,cluster_lat,cluster_lon,cluster_persons_surveyed,cluster_annual_consumption_pc,cluster_annual_phone_consumption_pc,cluster_cellphones_pc,cluster_estimated_annual_phone_cost_pc,cluster_annual_consumption_hh_na,cluster_annual_phone_consumption_hh_na,cluster_cellphones_ph_na,cluster_estimated_annual_phone_cost_ph_na,cluster_num_hh_surveyed,cluster_nightlights
0,-17.095150,35.217213,79,961.328026,47.627469,0.177215,428.481013,0.0,0.0,0.0,0.5000,16,0.000000
1,-17.092351,35.114643,70,855.258482,3.189638,0.028571,32.571429,0.0,0.0,0.0,0.8750,16,0.000000
2,-17.016698,35.079629,78,1058.343450,1.978659,0.025641,19.230769,0.0,0.0,0.0,0.8750,16,0.000000
3,-16.977243,35.205706,66,1127.493134,8.631155,0.045455,83.333333,0.0,0.0,0.0,0.8125,16,0.121212
4,-16.956385,35.168967,61,736.167585,5.081308,0.065574,49.180328,0.0,0.0,0.0,0.7500,16,0.502674


In [34]:
# This is a bunch of code from the Jean et al Github that is modified to work with Python3 and our data

import numpy as np
import pandas as pd
import random
from scipy import stats
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
import sklearn.linear_model as linear_model
import matplotlib.pyplot as plt
from matplotlib.collections import EllipseCollection
import seaborn as sns
import joblib
from sklearn import metrics
import pickle

def predict_consumption(
    X, y, k=5, k_inner=5, points=10,
        alpha_low=1, alpha_high=5, margin=0.25):
    """
    Plots predicted consumption
    """
    y_hat, r2, ridges, scalers = run_cv(X, y, k, k_inner, points, alpha_low, alpha_high)
    return y_hat, r2, ridges, scalers


def run_cv(X, y, k, k_inner, points, alpha_low, alpha_high, randomize=False):
    """
    Runs nested cross-validation to make predictions and compute r-squared.
    """
    alphas = np.logspace(alpha_low, alpha_high, points)
    r2s = np.zeros((k,))
    y_hat = np.zeros_like(y)
    kf = KFold(n_splits=k, shuffle=True)
    fold = 0
    ridges = []
    scalers = []
    for train_idx, test_idx in kf.split(X):
        r2s, y_hat, fold, ridge, scaler = evaluate_fold(
            X, y, train_idx, test_idx, k_inner, alphas, r2s, y_hat, fold,
            randomize)
        ridges.append(ridge)
        scalers.append(scaler)
    return y_hat, r2s.mean(), ridges, scalers


def scale_features(X_train, X_test):
    """
    Scales features using StandardScaler.
    """
    X_scaler = StandardScaler(with_mean=True, with_std=False)
    X_train = X_scaler.fit_transform(X_train)
    X_test = X_scaler.transform(X_test)
    return X_train, X_test, X_scaler


def train_and_predict_ridge(alpha, X_train, y_train, X_test):
    """
    Trains ridge model and predicts test set.
    """
    ridge = linear_model.Ridge(alpha)
    ridge.fit(X_train, y_train)
    y_hat = ridge.predict(X_test)
    return y_hat, ridge


def predict_inner_test_fold(X, y, y_hat, train_idx, test_idx, alpha):
    """
    Predicts inner test fold.
    """
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    X_train, X_test, _ = scale_features(X_train, X_test)
    y_hat[test_idx], _ = train_and_predict_ridge(alpha, X_train, y_train, X_test)
    return y_hat


def find_best_alpha(X, y, k_inner, alphas):
    """
    Finds the best alpha in an inner CV loop.
    """
    kf = KFold(n_splits=k_inner, shuffle=True)
    best_alpha = 0
    best_r2 = 0
    for idx, alpha in enumerate(alphas):
        y_hat = np.zeros_like(y)
        for train_idx, test_idx in kf.split(X):
            y_hat = predict_inner_test_fold(
                X, y, y_hat, train_idx, test_idx, alpha)
        r2 = stats.pearsonr(y, y_hat)[0] ** 2
        if r2 > best_r2:
            best_alpha = alpha
            best_r2 = r2
    print('best alpha', best_alpha)
    return best_alpha


def evaluate_fold(
    X, y, train_idx, test_idx, k_inner, alphas, r2s, y_hat, fold,
        randomize):
    """
    Evaluates one fold of outer CV.
    """
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]
    if randomize:
        random.shuffle(y_train)
    best_alpha = find_best_alpha(X_train, y_train, k_inner, alphas)
    X_train, X_test, scaler = scale_features(X_train, X_test)
    y_test_hat, ridge = train_and_predict_ridge(best_alpha, X_train, y_train, X_test)
    r2 = stats.pearsonr(y_test, y_test_hat)[0] ** 2
    r2s[fold] = r2
    y_hat[test_idx] = y_test_hat
    return r2s, y_hat, fold + 1, ridge, scaler


In [10]:
y = clusters['cluster_annual_consumption_pc'].values
y_log = np.log(y + 0.0001)

In [11]:
y_hat_log, r2, ridges, scalers = predict_consumption(feats, y_log)
r2

best alpha 10.0
best alpha 77.4263682681127
best alpha 215.44346900318823
best alpha 77.4263682681127
best alpha 10.0


0.4532365159422239

In [12]:
y_hat, r2, ridges, scalers = predict_consumption(feats, y)
r2

best alpha 215.44346900318823
best alpha 215.44346900318823
best alpha 599.4842503189409
best alpha 599.4842503189409
best alpha 599.4842503189409


0.2676335417621222

In [13]:
clusters['predicted_cluster_annual_consumption_pc'] = y_hat
clusters['predicted_cluster_log_cons'] = y_hat_log

In [14]:
re = RidgeEnsemble(ridges, scalers)
metrics.r2_score(y, re.predict(feats))

0.15490207052283644

In [17]:
joblib.dump(re, '../models/ridge_consumption.joblib')

['../models/ridge_consumption.joblib']

In [22]:
y = clusters['cluster_annual_phone_consumption_pc'].values
y_log = np.log(y + 0.0001)

In [23]:
y_hat_log, r2, ridges, scalers = predict_consumption(feats, y_log)
r2

best alpha 215.44346900318823
best alpha 215.44346900318823
best alpha 599.4842503189409
best alpha 215.44346900318823
best alpha 215.44346900318823


0.1942243389251287

In [24]:
y_hat, r2, ridges, scalers = predict_consumption(feats, y)
r2

best alpha 77.4263682681127
best alpha 77.4263682681127
best alpha 77.4263682681127
best alpha 77.4263682681127
best alpha 77.4263682681127


0.3752444534237157

In [25]:
clusters['predicted_cluster_annual_phone_consumption_pc'] = y_hat
clusters['predicted_cluster_log_annual_phone_consumption_pc'] = y_hat_log

In [27]:
re = RidgeEnsemble(ridges, scalers)
metrics.r2_score(y, re.predict(feats))

0.41915520039230636

In [28]:
joblib.dump(re, '../models/ridge_phone_consumption.joblib')

['../models/ridge_phone_consumption.joblib']

In [29]:
y = clusters['cluster_cellphones_pc'].values
y_log = np.log(y + 0.0001)

In [35]:
y_hat_log, r2, ridges, scalers = predict_consumption(feats, y_log)
r2

best alpha 215.44346900318823
best alpha 77.4263682681127
best alpha 77.4263682681127
best alpha 27.825594022071243
best alpha 10.0


0.25008891035688213

In [36]:
y_hat, r2, ridges, scalers = predict_consumption(feats, y)
r2

best alpha 27.825594022071243
best alpha 77.4263682681127
best alpha 27.825594022071243
best alpha 77.4263682681127
best alpha 27.825594022071243


0.5033864791415932

In [37]:
clusters['predicted_cluster_cellphones_pc'] = y_hat
clusters['predicted_cluster_log_cellphones_pc'] = y_hat_log

In [38]:
re = RidgeEnsemble(ridges, scalers)
metrics.r2_score(y, re.predict(feats))

0.5705474211027788

In [40]:
joblib.dump(re, '../models/ridge_phone_density.joblib')

['../models/ridge_phone_density.joblib']

In [41]:
not_nas = ~clusters['cluster_estimated_annual_phone_cost_pc'].isna()

In [42]:
y = clusters['cluster_estimated_annual_phone_cost_pc'].values[not_nas]
y_log = np.log(y + 0.0001)

In [44]:
y_hat_log, r2, ridges, scalers = predict_consumption(feats[not_nas], y_log)
r2

best alpha 77.4263682681127
best alpha 77.4263682681127
best alpha 599.4842503189409
best alpha 599.4842503189409
best alpha 27.825594022071243


0.17912792993120533

In [45]:
y_hat_log, r2, ridges, scalers = predict_consumption(feats[not_nas], y)
r2

best alpha 215.44346900318823
best alpha 27.825594022071243
best alpha 77.4263682681127
best alpha 27.825594022071243
best alpha 599.4842503189409


0.3359779551009823

In [46]:
clusters['predicted_cluster_estimated_annual_phone_cost_pc'] = np.nan
clusters['predicted_cluster_log_estimated_annual_phone_cost_pc'] = np.nan

clusters['predicted_cluster_estimated_annual_phone_cost_pc'].loc[not_nas] = y_hat
clusters['predicted_cluster_log_estimated_annual_phone_cost_pc'].loc[not_nas] = y_hat_log

/Applications/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [47]:
clusters.head()

,cluster_lat,cluster_lon,cluster_persons_surveyed,cluster_annual_consumption_pc,cluster_annual_phone_consumption_pc,cluster_cellphones_pc,cluster_estimated_annual_phone_cost_pc,cluster_annual_consumption_hh_na,cluster_annual_phone_consumption_hh_na,cluster_cellphones_ph_na,...,cluster_num_hh_surveyed,cluster_nightlights,predicted_cluster_annual_consumption_pc,predicted_cluster_log_cons,predicted_cluster_annual_phone_consumption_pc,predicted_cluster_log_annual_phone_consumption_pc,predicted_cluster_cellphones_pc,predicted_cluster_log_cellphones_pc,predicted_cluster_estimated_annual_phone_cost_pc,predicted_cluster_log_estimated_annual_phone_cost_pc
0,-17.095150,35.217213,79,961.328026,47.627469,0.177215,428.481013,0.0,0.0,0.0,...,16,0.000000,1241.247367,6.850958,29.121409,2.830344,0.144066,-2.271444,0.144066,218.328612
1,-17.092351,35.114643,70,855.258482,3.189638,0.028571,32.571429,0.0,0.0,0.0,...,16,0.000000,994.290808,6.982761,23.046583,2.912264,0.093851,-3.010220,0.093851,330.626006
2,-17.016698,35.079629,78,1058.343450,1.978659,0.025641,19.230769,0.0,0.0,0.0,...,16,0.000000,1026.633617,6.969120,20.632943,2.766274,0.064562,-2.587164,0.064562,170.094968
3,-16.977243,35.205706,66,1127.493134,8.631155,0.045455,83.333333,0.0,0.0,0.0,...,16,0.121212,1545.520360,7.137436,45.898687,2.962992,0.140639,-2.223968,0.140639,359.515688
4,-16.956385,35.168967,61,736.167585,5.081308,0.065574,49.180328,0.0,0.0,0.0,...,16,0.502674,1482.647091,6.914620,28.870149,2.611013,0.101224,-2.508362,0.101224,270.882969


In [52]:
to_drop = []
for c in clusters.columns:
    if '_na' in c:
        to_drop.append(c)

clusters.drop(to_drop, axis=1).to_csv('cluster_cnn_predictions.csv', index=False)
